# Pipeline SWV → Ganhos de Controle (Explicado)

Este notebook documenta, com fórmulas e exemplos, o fluxo dos seus scripts:

1) Exportação SWV e sanitização (`filter_swv_export.py`)\
2) Séries derivadas e métricas (`analyze_step_responses.py`)\
3) Identificação do modelo FOPDT\
4) Síntese dos ganhos PD (Ziegler–Nichols, curva de reação)\
5) Geração do summary e CSVs derivados\
6) Uso dos perfis consolidados por `tuning_profiles.py`

Ao final há células de código que demonstram as fórmulas e leitura de amostras.

## 1. Exportação SWV e Sanitização

Os dumps SWV/ITM podem conter ruído textual ou linhas truncadas. O script `filter_swv_export.py` padroniza cada arquivo em um CSV com linhas válidas no formato:

```
axis,id,time_ms,encoder,pulses
```

Regras de consistência aplicadas (por arquivo):
- Aceita apenas linhas numéricas que casam a regex `^\s*(-?\d+),(-?\d+),(-?\d+),(-?\d+),(-?\d+)\s*$`.
- O primeiro `axis` numérico define o eixo do arquivo; todas as linhas seguintes com outro valor são descartadas.
- `id` precisa ser estritamente crescente (cada linha nova: `seq > last_id`).
- `time_ms` e `pulses` não podem andar para trás (`t_ms >= last_time` e `pulses >= last_pulses`).
- O valor de `encoder` é tomado em módulo (sempre positivo).
- Arquivos que não preservam nenhuma linha são descartados (arquivo de saída apagado).

Resultado: `*_filtered.csv` sob `CNC_Controller/SWV_export/` (ou diretório indicado).

## 2. Séries Derivadas (Velocidades)

A partir de `axis,id,time_ms,encoder,pulses`, calculam-se velocidades por diferenças finitas. Há dois caminhos no script de análise:

- Série bruta: usa pares adjacentes (linha `k` e `k-1`) para computar intervalos `Δt_k`. Linhas com `Δt_k <= 0` são ignoradas.
- Série derivada com janela deslizante (stride): usa a primeira e a última linha de uma janela de `N` amostras válidas para reduzir ruído.

Definições (com tempo em segundos):

$\displaystyle t_k = 	frac{time\_ms(k)}{1000}$\
$\displaystyle \Delta t_k = t_k - t_{k-N+1}$ (janela)\
$\displaystyle v_{cmd}(k) = 	frac{pulses(k) - pulses(k-N+1)}{\Delta t_k}$\
$\displaystyle v_{enc}(k) = 	frac{encoder(k) - encoder(k-N+1)}{\Delta t_k}$

O tamanho da janela `N` (parâmetro `--stride`, padrão 10) suaviza as derivadas preservando a detecção do degrau.


## 3. Identificação FOPDT

Assume-se o modelo de primeira ordem com atraso (FOPDT) para a malha de velocidade:

$\displaystyle G_v(s) = rac{K \; e^{-Ls}}{\tau s + 1}$

- Ganho estático: $K = \tfrac{v_{enc,ss}}{v_{cmd,ss}}$, onde `ss` é valor médio no regime estacionário (janela final).
- Atraso puro: $L$ é estimado pela diferença entre os instantes em que `v_cmd` e `v_enc` cruzam 5% dos seus respectivos valores de regime: $L = t_{enc}^{5\%} - t_{cmd}^{5\%}$.
- Constante de tempo: $\tau$ usa o cruzamento a 63,2% de `v_enc`: $t_{63} =$ primeiro instante em que `v_enc(t) \ge 0,632 \cdot v_{enc,ss}` e $\tau = t_{63} - L$.

Tratamento de borda e resolução temporal:
- Se $L \le 0$ ou indefinido, usa-se a menor `Δt` observada nos dados (`min_dt_raw`) como piso para $L$.
- Se $\tau \le 0$ ou indefinido, igualmente usa-se `min_dt_raw`.
- Valores inválidos (NaN/inf) são descartados.

Classificação de forma do degrau (opcional): compara o tempo até 90% com a duração total; rotula *step-like* se a subida for curta (≤ 15% do total) ou *ramp-like* caso contrário.

## 4. Síntese de Ganhos (Ziegler–Nichols, Curva de Reação)

O controlador usado aqui é PD na malha de velocidade (Ki = 0, pois a malha de posição já integra). As regras de Z–N para curva de reação aplicadas são:

$\displaystyle K_p = 1.2 \cdot rac{\tau}{K \cdot L}$\
$\displaystyle T_d = 0.5 \cdot L$\
$\displaystyle K_d = K_p \cdot T_d$\

Sugestão segura: usar inicialmente $K_p/2$ e aumentar gradualmente até $K_p$ caso a resposta comporte-se bem.

Conversão para inteiros do firmware (escala `K_SCALE = 256`):\
$\displaystyle kp_i = \mathrm{round}(K_p \cdot 256)$,\
$\displaystyle ki_i = \mathrm{round}(K_i \cdot 256)$,\
$\displaystyle kd_i = \mathrm{round}(K_d \cdot 256)$.

## 5. Summary e CSVs Derivados

O script `analyze_step_responses.py` gera:
- Um relatório consolidado `analysis_summary.txt` sob `CNC_Controller/SWV_export`, contendo para cada arquivo analisado as métricas e os ganhos PD.
- (Opcional) CSVs derivados por arquivo (em `analysis_data/`), contendo `time_s`, `delta_t_s`, `vel_cmd_sps`, `vel_enc_sps` e outros campos úteis para inspeção/plot.

## 6. Integração com tuning_profiles.py

Os valores validados do summary alimentam uma tabela no `tuning_profiles.py` associada a pares (eixo, microstep). Cada entrada guarda $(K_p, K_i, K_d, L, \tau)$ e os valores de regime. Os simuladores (`fopdt_simulator.py`, `interactive_sim.py`) consomem essa tabela e convertem para inteiros de firmware com a escala 256.

In [ ]:
# Utilitários demonstrativos das fórmulas usadas no pipeline
from __future__ import annotations
from pathlib import Path
import csv, math
from typing import List, Dict, Optional, Tuple

def load_filtered_rows(csv_path: Path) -> List[Dict[str, int]]:
    rows: List[Dict[str, int]] = []
    with csv_path.open('r', encoding='utf-8', newline='') as fh:
        reader = csv.reader(fh)
        for cols in reader:
            if len(cols) < 5:
                continue
            try:
                axis = int(cols[0]); seq = int(cols[1]); t_ms = int(cols[2])
                encoder = int(cols[3]); pulses = int(cols[4])
            except ValueError:
                continue
            rows.append({'axis': axis, 'seq': seq, 'time_ms': t_ms, 'encoder': encoder, 'pulses': pulses})
    return rows

def compute_raw_series(rows: List[Dict[str, int]]):
    T: List[float] = []
    Vc: List[float] = []
    Ve: List[float] = []
    dt_vals: List[float] = []
    prev = None
    for r in rows:
        if prev is None:
            prev = r; continue
        dt_ms = r['time_ms'] - prev['time_ms']
        if dt_ms <= 0:
            prev = r; continue
        dt = dt_ms / 1000.0
        dp = r['pulses'] - prev['pulses']
        de = r['encoder'] - prev['encoder']
        Vc.append(dp / dt if dt != 0 else math.nan)
        Ve.append(de / dt if dt != 0 else math.nan)
        T.append(r['time_ms'] / 1000.0)
        dt_vals.append(dt)
        prev = r
    return T, Vc, Ve, dt_vals

def steady_value(values: List[float], frac: float = 0.2) -> Optional[float]:
    if not values:
        return None
    start = max(0, len(values) - max(1, int(len(values) * frac)))
    window = [v for v in values[start:] if (v is not None and not math.isnan(v))]
    if not window:
        window = [v for v in values if (v is not None and not math.isnan(v))]
    if not window:
        return None
    return sum(window) / len(window)

def first_crossing(times: List[float], values: List[float], thr: float) -> Optional[float]:
    if thr is None:
        return None
    for t, v in zip(times, values):
        if v is None or math.isnan(v):
            continue
        if v >= thr:
            return t
    return None

def identify_fopdt(times: List[float], Vc: List[float], Ve: List[float], dt_vals: List[float]):
    steady_cmd = steady_value(Vc)
    steady_enc = steady_value(Ve)
    K = (steady_enc / steady_cmd) if steady_cmd and steady_cmd > 0 else None
    cmd_thr = (steady_cmd * 0.05) if steady_cmd and steady_cmd > 0 else None
    enc_thr = (steady_enc * 0.05) if steady_enc and steady_enc > 0 else None
    t_cmd = first_crossing(times, Vc, cmd_thr) if cmd_thr else None
    t_enc = first_crossing(times, Ve, enc_thr) if enc_thr else None
    L = (t_enc - t_cmd) if (t_cmd is not None and t_enc is not None) else None
    if L is not None and L < 0: L = 0.0
    t63_target = (steady_enc * 0.632) if steady_enc else None
    t63 = first_crossing(times, Ve, t63_target) if t63_target else None
    tau = (t63 - L) if (t63 is not None and L is not None) else None
    if tau is not None and tau < 0: tau = None
    min_dt = min(dt_vals) if dt_vals else None
    if (L is None or L <= 0) and min_dt: L = min_dt
    if (tau is None or tau <= 0) and min_dt: tau = min_dt
    return K, L, tau, steady_cmd, steady_enc

def zn_pd(K: Optional[float], L: Optional[float], tau: Optional[float]):
    if not (K and L and tau) or (K <= 0 or L <= 0 or tau <= 0):
        return None, 0.0, None
    Kp = 1.2 * (tau / (K * L))
    Td = 0.5 * L
    Kd = Kp * Td
    return Kp, 0.0, Kd

def firmware_gains(kp: float, ki: float, kd: float, k_scale: int = 256) -> Tuple[int,int,int]:
    return (int(round(kp * k_scale)), int(round(ki * k_scale)), int(round(kd * k_scale)))

print('Funções carregadas.')


In [ ]:
# Demonstração: tenta carregar o primeiro *_filtered.csv e calcular K, L, tau e ganhos PD
base = Path.cwd() / 'CNC_Controller' / 'SWV_export'
candidates = sorted(base.glob('*_filtered.csv')) if base.exists() else []
if not candidates:
    print('Nenhum *_filtered.csv encontrado sob CNC_Controller/SWV_export — rode filter_swv_export.py e analyze_step_responses.py.')
else:
    sample = candidates[0]
    print('Exemplo:', sample.name)
    rows = load_filtered_rows(sample)
    T, Vc, Ve, dt_vals = compute_raw_series(rows)
    K, L, tau, Vc_ss, Ve_ss = identify_fopdt(T, Vc, Ve, dt_vals)
    Kp, Ki, Kd = zn_pd(K, L, tau)
    print(f'K={K}, L={L}, tau={tau}')
    if Kp is not None:
        print(f'Z–N PD: Kp={Kp:.4f}, Ki={Ki:.4f}, Kd={Kd:.4f}')
        print('Firmware ints:', firmware_gains(Kp, Ki, Kd))
    else:
        print('Não foi possível sintetizar ganhos (dados insuficientes).')


## Como reproduzir pelo terminal

1. Sanitizar: `python filter_swv_export.py`\
2. Analisar: `python analyze_step_responses.py -v`\
3. Simular: `python fopdt_simulator.py --axes X:256,Y:16,Z:256 --plot --emit-fw`\
4. Atualizar `tuning_profiles.py` com os novos valores, se desejar consolidá-los.